In [1]:
import cv2
import glob
import numpy as np
from skimage.transform import resize
from keras.utils import to_categorical

img_length = 60

def process_img(img_name):
    img = cv2.imread(img_name)
    
    mask = np.zeros(img.shape[:2], np.uint8)
    rect = (0, 0, img_length, img_length)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    img = img * mask2[:, :, np.newaxis]
    
    resized_img = resize(img, (img_length, img_length, 3), preserve_range=True)
    return np.array(resized_img).astype(int)

full_thickness_img_names = glob.glob('./images/full_thickness/*.jpg') + glob.glob('./images/full_thickness/*.JPG')
X1 = [process_img(img_name) for img_name in full_thickness_img_names]
y1 = [0] * len(X1)

partial_thickness_img_names = glob.glob('./images/partial_thickness/*.jpg') + glob.glob('./images/partial_thickness/*.JPG')
X2 = [process_img(img_name) for img_name in partial_thickness_img_names]
y2 = [1] * len(X2)

superficial_dermal_img_names = glob.glob('./images/superficial_dermal/*.jpg') + glob.glob('./images/superficial_dermal/*.JPG')
X3 = [process_img(img_name) for img_name in superficial_dermal_img_names]
y3 = [2] * len(X3)

X = np.concatenate((X1, X2, X3))

y = np.concatenate((y1, y2, y3))
y = to_categorical(y)

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = img_length, img_length

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=20, epochs=100, verbose=1)

train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print('Train Accuracy: {}'.format(train_acc))

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: {}'.format(test_acc))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
44/44 [==============================] - 0s 10ms/step - loss: 4.9659 - accuracy: 0.3409
Epoch 2/100
44/44 [==============================] - 0s 5ms/step - loss: 4.6832 - accuracy: 0.4091
Epoch 3/100
44/44 [==============================] - 0s 6ms/step - loss: 4.8058 - accuracy: 0.4318
Epoch 4/100
44/44 [==============================] - 0s 5ms/step - loss: 3.7721 - accuracy: 0.5682
Epoch 5/100
44/44 [==============================] - 0s 5ms/step - loss: 4.1397 - accuracy: 0.4773
Epoch 6/100
44/44 [==============================] - 0s 5ms/step - loss: 3.8732 - accuracy: 0.5000
Epoch 7/100
44/44 [==============================] - 0s 6ms/step - loss: 4.4889 - accuracy: 0.4545
Epoch 8/100
44/44 [==============================] - 0s 5ms/step - loss: 4.4905 - accuracy: 0.5227
Epoch 9/100
44/44 [==============================] - 0s 5ms/step - loss: 4.6246 - accur

44/44 [==============================] - 0s 5ms/step - loss: 0.3121 - accuracy: 0.8636
Epoch 78/100
44/44 [==============================] - 0s 5ms/step - loss: 0.3097 - accuracy: 0.7727
Epoch 79/100
44/44 [==============================] - 0s 5ms/step - loss: 0.3530 - accuracy: 0.7955
Epoch 80/100
44/44 [==============================] - 0s 5ms/step - loss: 0.2995 - accuracy: 0.8409
Epoch 81/100
44/44 [==============================] - 0s 4ms/step - loss: 0.2854 - accuracy: 0.8409
Epoch 82/100
44/44 [==============================] - 0s 4ms/step - loss: 0.2991 - accuracy: 0.8409
Epoch 83/100
44/44 [==============================] - 0s 5ms/step - loss: 0.3166 - accuracy: 0.7727
Epoch 84/100
44/44 [==============================] - 0s 7ms/step - loss: 0.2747 - accuracy: 0.8182
Epoch 85/100
44/44 [==============================] - 0s 5ms/step - loss: 0.2873 - accuracy: 0.8864
Epoch 86/100
44/44 [==============================] - 0s 4ms/step - loss: 0.3084 - accuracy: 0.8182
Epoch 87/100
